In [1]:
import pandas as pd

In [2]:
columns = ['order_id', 'order_date', 'order_customer_id', 'order_status']

In [3]:
import json

In [5]:
schemas = json.load(open('../../data/retail_db/schemas.json'))

In [7]:
sorted(schemas['orders'], key=lambda col:col['column_position'])

[{'column_name': 'order_id', 'data_type': 'integer', 'column_position': 1},
 {'column_name': 'order_date', 'data_type': 'string', 'column_position': 2},
 {'column_name': 'order_customer_id',
  'data_type': 'timestamp',
  'column_position': 3},
 {'column_name': 'order_status', 'data_type': 'string', 'column_position': 4}]

In [8]:
columns = [col['column_name'] for col in sorted(schemas['orders'], key=lambda col:col['column_position'])]

In [9]:
columns

['order_id', 'order_date', 'order_customer_id', 'order_status']

In [10]:
pd.read_csv?

In [11]:
orders = pd.read_csv('../../data/retail_db/orders/part-00000', names=columns)

In [12]:
orders

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE
...,...,...,...,...
68878,68879,2014-07-09 00:00:00.0,778,COMPLETE
68879,68880,2014-07-13 00:00:00.0,1117,COMPLETE
68880,68881,2014-07-19 00:00:00.0,2518,PENDING_PAYMENT
68881,68882,2014-07-22 00:00:00.0,10000,ON_HOLD


In [13]:
type(orders)

pandas.core.frame.DataFrame

In [15]:
orders.shape

(68883, 4)

In [16]:
daily_status_count = orders.\
    groupby(['order_date', 'order_status'])['order_id'].\
    agg(order_count='count').\
    reset_index()

In [17]:
daily_status_count

,order_date,order_status,order_count
0,2013-07-25 00:00:00.0,CANCELED,1
1,2013-07-25 00:00:00.0,CLOSED,20
2,2013-07-25 00:00:00.0,COMPLETE,42
3,2013-07-25 00:00:00.0,ON_HOLD,5
4,2013-07-25 00:00:00.0,PAYMENT_REVIEW,3
...,...,...,...
3198,2014-07-24 00:00:00.0,PAYMENT_REVIEW,2
3199,2014-07-24 00:00:00.0,PENDING,23
3200,2014-07-24 00:00:00.0,PENDING_PAYMENT,37
3201,2014-07-24 00:00:00.0,PROCESSING,20


In [19]:
daily_status_count.to_sql?

In [20]:
host = '35.223.141.206'
port = 5432
database = 'aprendiendo'
user = 'aprendiendo_user'
password = 'qwerty'

In [21]:
conn_uri = f'postgresql://{user}:{password}@{host}:{port}/{database}'

In [22]:
daily_status_count.to_sql(
    'daily_status_count',
    conn_uri,
    if_exists='replace',
    index=False
)

203

In [23]:
pd.read_sql?

In [24]:
dsc = pd.read_sql(
    'daily_status_count',
    conn_uri
)

In [25]:
dsc.shape

(3203, 3)

In [26]:
dsc

,order_date,order_status,order_count
0,2013-07-25 00:00:00.0,CANCELED,1
1,2013-07-25 00:00:00.0,CLOSED,20
2,2013-07-25 00:00:00.0,COMPLETE,42
3,2013-07-25 00:00:00.0,ON_HOLD,5
4,2013-07-25 00:00:00.0,PAYMENT_REVIEW,3
...,...,...,...
3198,2014-07-24 00:00:00.0,PAYMENT_REVIEW,2
3199,2014-07-24 00:00:00.0,PENDING,23
3200,2014-07-24 00:00:00.0,PENDING_PAYMENT,37
3201,2014-07-24 00:00:00.0,PROCESSING,20


In [27]:
dsc = pd.read_sql(
    '''
        SELECT order_status, sum(order_count) AS order_count FROM daily_status_count
        GROUP BY 1
        ORDER BY 2 DESC
    ''',
    conn_uri
)

In [28]:
dsc

,order_status,order_count
0,COMPLETE,22899.0
1,PENDING_PAYMENT,15030.0
2,PROCESSING,8275.0
3,PENDING,7610.0
4,CLOSED,7556.0
5,ON_HOLD,3798.0
6,SUSPECTED_FRAUD,1558.0
7,CANCELED,1428.0
8,PAYMENT_REVIEW,729.0
